In [1]:
# Requesting and receiving data from Arduino v1

"""
Code based on code from:
https://be189.github.io/lessons/10/control_of_arduino_with_python.html
https://be189.github.io/lessons/11/control_with_buttons.html
"""

# NOTE:
# This will not work if the Serial Monitor or Serial Plotter of the Arduino IDE open
# They keep the port busy so Python cannot communicate with Arduino

# -------------
# Imports
# -------------
import re
import asyncio
import time

import numpy as np
import pandas as pd

import serial
import serial.tools.list_ports

import bokeh.plotting
import bokeh.io
import bokeh.driving

# ---------------------------------
# Defining consts and magic numbers
# ---------------------------------
# URL of Juptyer notebook, should be default "localhost:8888"
NOTEBOOK_URL = "localhost:8888"
# The following should match the consts of the same name in the Arduino code
# Handshake consts
HANDSHAKE = 0
TEMP_REQUEST = 1
ON_REQUEST = 2
STREAM = 3
READ_DAQ_DELAY = 4
# Rate at which information is transferred in a communication channel
BAUDRATE = 9600
# Delay before data recording starts, to give MAX6675 chips time to stabilize
DELAY = 0 # 250

In [2]:
# -------------
# Preliminaries
# -------------
bokeh.io.output_notebook()

# ---------
# Functions
# ---------
"""
Many of these functions are heavily borrowed from BE/EE/MedE 189
https://be189.github.io/index.html
"""

def find_arduino(port=None):
    """
    Either returns the port input to it as a parameter, or searchs for a port
    Note that on Windows devices (and potentially others as well),
    the manufacturer could not appear as 'Arduino',
    which messes up the functions ability to find the port
    For Windows, the port should be input at something like 'COM3'
    
    You can see/select the port in the Arduino IDE under
    Tools > Port: ...
    """
    # Check if a port has been input
    if port is None:
        ports = serial.tools.list_ports.comports()
        for p in ports:
            # Use manufacturer to check which port is the Ardunio port
            if p.manufacturer is not None and "Arduino" in p.manufacturer:
                port = p.device
        if port is None:
            raise Exception("Port not found. Double check that the Arduino is plugged in before further troublshooting.")
    return port

def handshake_arduino(arduino, sleep_time=DELAY, print_handshake_message=False, handshake_code=HANDSHAKE):
    """
    Performs handshake with Ardunio
    """
    # Check connection is established by sending and reciving bytes
    print("Checking if connection is established by sending and reciving bytes")
    arduino.close()
    arduino.open()
    
    # Wait for MAX6675 chips to stabilize
    print("Waiting for MAX6675 chips to stabilize")
    time.sleep(sleep_time)
    print("Stabilization complete")

    # Set a long timeout to complete handshake
    print("Setting a long timeout to complete handshake")
    timeout = arduino.timeout
    arduino.timeout = 2

    # Read and discard everything that may be in the input buffer
    print("Reading and discarding everything that may be in the input buffer")
    _ = arduino.read_all()

    # Send request to Arduino
    print("Sending request to Arduino")
    arduino.write(bytes([handshake_code]))

    # Read in what Arduino sent
    print("Reading in what Arduino sent")
    handshake_message = arduino.read_until()

    # Send and receive request again
    print("Sending and receiving request again")
    arduino.write(bytes([handshake_code]))
    handshake_message = arduino.read_until()

    # Print the handshake message, if desired
    if print_handshake_message:
        print("Handshake message: " + handshake_message.decode())

    # Reset the timeout
    arduino.timeout = timeout
    
def parse_raw(raw):
    """
    Parse data from Arduino separated by ","
    """
    print(raw)
    raw = raw.decode()
    if raw[-1] != "\n":
        raise ValueError(
            "Input must end with newline, otherwise message is incomplete."
        )
    t, T1, T2 = raw.rstrip().split(",")
    return int(t), float(T1), float(T2)

def daq_stream(arduino, n_data=100, delay=20, time_out=100):
    """
    Obtain `n_data` data points from an Arduino stream
    with a delay of `delay` milliseconds between each
    """
    # Specify delay
    arduino.write(bytes([READ_DAQ_DELAY]) + (str(delay) + "x").encode())

    # Initialize output
    time_ms = np.empty(n_data)
    temp1 = np.empty(n_data)
    temp2 = np.empty(n_data)

    # Turn on the stream
    arduino.write(bytes([STREAM]))

    # Receive data
    # i counts successful data reading attempts
    i = 0
    # j counts total attempts to read data
    j = 0
    while i < n_data and j < time_out:
        raw = arduino.read_until()

        # print(parse_raw_temp(raw))
        
        try:
            t, T1, T2 = parse_raw(raw)
            time_ms[i] = t
            temp1[i] = T1
            temp2[i] = T2
            i += 1
        except:
            print("Error: parse_raw failed")
            pass
        j += 1

    # Turn off the stream
    arduino.write(bytes([ON_REQUEST]))

    return pd.DataFrame({'time (ms)': time_ms, 'temp1 (C)': temp1, 'temp2 (C)': temp2})

Loading BokehJS ...

In [3]:
# -----------------
# Calling functions
# -----------------
# Get port
print("Finding port")
port = find_arduino()
# Open port
print("Opening port")
arduino = serial.Serial(port, baudrate=9600, timeout=1)
print("Establishing handshake")
# Make handshake
handshake_arduino(arduino, print_handshake_message=True)

df = daq_stream(arduino, n_data=10, delay=250, time_out=10)

df['time (sec)'] = df['time (ms)'] / 1000

print(df)

# p = bokeh.plotting.figure(
#     x_axis_label='time (s)',
#     y_axis_label='temp (T)',
#     frame_height=175,
#     frame_width=500,
#     x_range=[df['time (sec)'].min(), df['time (sec)'].max()],
# )
# p.line(source=df, x='time (sec)', y='temp1 (T)')

# bokeh.io.show(p)

# Closing connection with Arduino
arduino.close()

Finding port
Opening port
Establishing handshake
Checking if connection is established by sending and reciving bytes
Waiting for MAX6675 chips to stabilize
Stabilization complete
Setting a long timeout to complete handshake
Reading and discarding everything that may be in the input buffer
Sending request to Arduino
Reading in what Arduino sent
Sending and receiving request again
Handshake message: Message received.

b'1031,17.7500000000,14.2500000000\r\n'
b'1282,18.2500000000,13.7500000000\r\n'
b'1533,17.5000000000,13.5000000000\r\n'
b'1784,17.2500000000,13.7500000000\r\n'
b'2035,17.7500000000,14.0000000000\r\n'
b'2286,17.7500000000,13.7500000000\r\n'
b'2537,18.0000000000,13.7500000000\r\n'
b'2788,18.0000000000,13.5000000000\r\n'
b'3039,17.7500000000,13.7500000000\r\n'
b'3290,18.0000000000,13.7500000000\r\n'
   time (ms)  temp1 (C)  temp2 (C)  time (sec)
0     1031.0      17.75      14.25       1.031
1     1282.0      18.25      13.75       1.282
2     1533.0      17.50      13.50     